In [2]:
pip install azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install azure-identity azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [4]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [5]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


#   List the datastores :

In [6]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

workspaceworkingdirectory
workspacefilestore
workspaceartifactstore
workspaceblobstore


## Create data assets

There are three types of data assets:



In [9]:
import pandas as pd

pd.read_csv('insurance.csv')

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [10]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = './insurance.csv'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="Data asset pointing to a local file, automatically uploaded to the default datastore",
    name="insurance-price"
)

ml_client.data.create_or_update(my_data)

Uploading insurance.csv (< 1 MB): 100%|██████████| 55.6k/55.6k [00:00<00:00, 3.42MB/s]




Data({'path': 'azureml://subscriptions/cda9116f-5326-4a9b-9407-bc3a4391c27c/resourcegroups/data_udemy/workspaces/dala_project/datastores/workspaceblobstore/paths/LocalUpload/84cb4d3be3254fa514c3958737574704/insurance.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'insurance-price', 'description': 'Data asset pointing to a local file, automatically uploaded to the default datastore', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/cda9116f-5326-4a9b-9407-bc3a4391c27c/resourceGroups/data_udemy/providers/Microsoft.MachineLearningServices/workspaces/dala_project/data/insurance-price/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/captgt0071/code/Users/captgt007/data', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f8605920280>,

# To verify that the new data assets have been created, you can list all data assets in the workspace again:

In [11]:
datasets = ml_client.data.list()
for ds_name in datasets:
    print(ds_name.name)

insurance-price


In [19]:
data = ml_client.data.get(name='insurance-price', version=1)
df = pd.read_csv(data.path)
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500
